In [1]:
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [2]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.14.0


In [3]:
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from Vanilla_90Window_25Stride_sub.model import model

In [4]:
training_data = np.load("training_data.npy").astype(np.float32)
categories = np.load("categories.npy")
valid_periods=np.load("valid_periods.npy")
categories_unique = np.unique(categories)

print(training_data.shape)
print(valid_periods.shape)
print(categories.shape)

(48000, 2776)
(48000, 2)
(48000,)


In [5]:
def build_dataset(category, window_length=200, telescope=18, from_start=True):
    dataset = []

    skipped_samples = 0
    dataset_window_length = window_length + telescope
    sample_map = categories==category
    
    valid_perios_for_category = valid_periods[sample_map]
    training_data_for_category = training_data[sample_map]

    for i in range(len(valid_perios_for_category)):
        from_time = valid_perios_for_category[i,0]
        to_time = valid_perios_for_category[i,1]
        if(to_time-from_time<dataset_window_length):
            skipped_samples+=1
            continue
        if(from_start):
            sample = training_data_for_category[i][from_time:from_time+dataset_window_length]
        else:
            sample = training_data_for_category[i][to_time-dataset_window_length:to_time]
        dataset.append(sample)

    dataset = np.array(dataset)
    dataset = dataset.reshape((dataset.shape[0], dataset.shape[1], 1))
    print("Skipped samples: ", skipped_samples)

    return dataset

In [6]:
window_length=200
telescope=9

In [7]:
datasets = {}
for category in categories_unique:
    dataset = build_dataset(category, window_length=window_length, telescope=telescope)
    datasets[category] = dataset

Skipped samples:  1891
Skipped samples:  7560
Skipped samples:  5099
Skipped samples:  4495
Skipped samples:  6807
Skipped samples:  178


In [8]:
dataset.shape

(99, 209, 1)

In [9]:
samples_per_category = 10

In [10]:
test_batch = np.array([])
labels = np.array([])
for category in categories_unique:
    dataset = datasets[category]
    indices = np.random.permutation(len(dataset))
    dataset = dataset[indices[:samples_per_category]]
    labels = np.concatenate((labels, np.array([category]*samples_per_category)))

    if(test_batch.shape[0]==0):
        test_batch = dataset
    else:
        test_batch = np.concatenate((test_batch, dataset))
        
test_batch.shape, labels.shape

((60, 209, 1), (60,))

In [11]:
X_batch = test_batch[:, :-telescope, :]
y_batch = test_batch[:, -telescope:, :]
X_batch.shape, y_batch.shape

((60, 200, 1), (60, 9, 1))

In [12]:
vanilla_90Window = model("Vanilla_90Window_25Stride_sub")
predictions = vanilla_90Window.predict(X_batch, labels)

1/1 [==============================] - 0s 18ms/step


In [14]:
predictions.shape
#Deve essere TensorShape([60, 9])

TensorShape([60, 9])

In [17]:
mse = ((predictions.numpy() - y_batch.reshape((60,9)))**2).mean()
mse

0.010535591